In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np

from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical

In [3]:
with open('text.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '')  # убираем первый невидимый символ

In [4]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                      lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [6]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('я', 21), ('притягиваю', 1), ('только', 21), ('плохое', 1), ('кому', 2), ('нужен', 2), ('с', 12), ('такой', 5), ('внешностью', 2), ('не', 91)]


In [7]:
data = tokenizer.texts_to_sequences([texts])
res = np.array( data[0] )

In [8]:
inp_words = 3
n = res.shape[0] - inp_words

In [10]:
X = np.array([res[i:i + inp_words] for i in range(n)])
Y = to_categorical(res[inp_words:], num_classes=maxWordsCount)

In [9]:
model = Sequential()
model.add(Embedding(maxWordsCount, 256, input_length = inp_words))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 256)            256000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               49280     
                                                                 
 dense (Dense)               (None, 1000)              129000    
                                                                 
Total params: 434,280
Trainable params: 434,280
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

history = model.fit(X, Y, batch_size=32, epochs=50)

Epoch 1/50
52/52 [==============================] - 2s 10ms/step - loss: 6.8544 - accuracy: 0.0355
Epoch 2/50
52/52 [==============================] - 1s 11ms/step - loss: 6.1887 - accuracy: 0.0559
Epoch 3/50
52/52 [==============================] - 1s 10ms/step - loss: 5.7683 - accuracy: 0.0817
Epoch 4/50
52/52 [==============================] - 1s 12ms/step - loss: 5.2657 - accuracy: 0.1262
Epoch 5/50
52/52 [==============================] - 1s 12ms/step - loss: 4.6191 - accuracy: 0.1971
Epoch 6/50
52/52 [==============================] - 1s 11ms/step - loss: 3.9349 - accuracy: 0.3191
Epoch 7/50
52/52 [==============================] - 1s 10ms/step - loss: 3.2809 - accuracy: 0.4489
Epoch 8/50
52/52 [==============================] - 1s 11ms/step - loss: 2.6965 - accuracy: 0.5769
Epoch 9/50
52/52 [==============================] - 1s 11ms/step - loss: 2.1810 - accuracy: 0.6791
Epoch 10/50
52/52 [==============================] - 1s 17ms/step - loss: 1.7499 - accuracy: 0.7584
Epoch 11/

In [12]:
def buildPhrase(texts, str_len=20):
    res = texts
    data = tokenizer.texts_to_sequences([texts])[0]
    for i in range(str_len):
        # x = to_categorical(data[i: i + inp_words], num_classes=maxWordsCount)  # преобразуем в One-Hot-encoding
        # inp = x.reshape(1, inp_words, maxWordsCount)
        x = data[i: i + inp_words]
        inp = np.expand_dims(x, axis=0)

        pred = model.predict(inp)
        indx = pred.argmax(axis=1)[0]
        data.append(indx)

        res += " " + tokenizer.index_word[indx]  # дописываем строку

    return res


res = buildPhrase("позитив добавляет годы")
print(res)

1/1 [==============================] - 0s 24ms/step
позитив добавляет годы вторым вашей жизни позитивность к вашему вокруг искорку в защитит глазах есть стрелы важны всегда вы можете поэтому позитивный выбор
